<a href="https://colab.research.google.com/github/hanklin3/18065/blob/main/6_7920_Fall_2024_Homework_5_problem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup



In this problem, we will consider the discounted stochastic LQ control problem, as described in Lecture 3. Specifically, we will consider using Q-learning to approximate the Q-function of this problem. Below, we provide an instant of this problem through the environment **LQEnv**. The environment is a python class, whose interface is very similar to that of OpenAI gym.

The code block below imports important and relevant packages, defnies the environment, and includes helper functions like a policy evaluation function.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import gym
from gym import error, spaces, utils
from gym.utils import seeding

dtype = np.float32
class LQEnv(gym.Env):
  def __init__(self, n, m, seed = 0, gamma = 0.9, sigma = 0.2):
    """
    The LQ environment, initialized with the parameters
    n: state dimension
    m: action dimension
    seed: random seed
    gamma: discount factor
    sigma: noise variance
    """
    np.random.seed(seed)
    self.n = n
    self.action_space = spaces.Box(low = -np.inf, high = np.inf, shape =  [m,1], dtype = dtype )
    self.observation_space = spaces.Box(low = -np.inf, high = np.inf, shape =  [n,1], dtype = dtype )
    self.A = np.array([[0.0488135 , 0.21518937],
                       [0.10276338, 0.04488318]
                        ])
    self.B =  np.array([[-0.0763452 ,  0.14589411],
                        [-0.06241279,  0.391773  ]
                       ])
    self.R =  np.array([[1.57567331, 0.96575621],
                        [0.96575621, 1.40655837]
                        ])
    self.Q =  np.array([[1.67940376, 0.12099823],
                        [0.12099823, 0.51263764]
                        ])
    self.state = self.observation_space.sample()
    self.sigma = sigma
    self.gamma = gamma

  def step(self, action):
    """
    This method is the primary interface between environment and agent.
    Paramters:
        action: array of shape [m,1]

    Returns:
        output: (next state:array, cost:float, done:bool, None)

    """
    err_msg = f"{action!r} ({type(action)}) invalid"
    assert self.action_space.contains(action), err_msg
    assert self.state is not None, "Call reset before using step method."
    w = self.sigma*np.random.randn()

    cost = (self.state.T.dot(self.Q)).dot(self.state) +  (action.T.dot(self.R)).dot(action)

    self.state = self.A.dot(self.state) + self.B.dot(action) + w

    done = False

    return self.state, cost, done, {}

  def reset(self, state = None):
    """
    This method resets the environment to its initial values.
    Paramters:
        state array
            set the env to specifc state (optional)
    Returns:
        observation:    array
                        the initial state of the environment
    """
    if state is None:
        # sample from a guassian with zero mean and std of 10
        self.state = 10*self.observation_space.sample()
    else:
        self.state = state
    return  self.state

  def close(self):
    """
    This method provides the user with the option to perform any necessary cleanup.
    """
    pass

  def sample_random_action(self):
      """
      sample actions from a normal dist with mean zero and std 10
      """
      return  10*env.action_space.sample()


def policy_evaluation(policy, env, T = 1000, N = 5):
    """
    This method evaluate the performance of a specifc policy on the env through simulating
    it on N trajectories each of length T.
    Paramters:
        policy: function that takes in state and return action
        env: instant of LQEnv
        T: number of iterations per trajectory
        N: number of trajectories
    Returns:
        output: mean discounted total cost

    """
    costs = []
    for ite in tqdm(range(N)):
        state = env.reset()
        gamma = env.gamma
        total_costs = 0
        for t in range(T):
            action = policy(state)
            state, cost, _, _ = env.step(action)
            total_costs += cost * gamma**(t)
        costs.append(total_costs)
    return np.mean(costs)

def lin_policy(K):
    """
    helper function to define linear policies of the form u = L@x
    """
    def policy(state):
        return K.dot(state)
    return policy


def Q_a(x,u, theta):
    """
    Q-function parameterized by the tuple/list theta
    """
    q =  x.T@theta[0].T@theta[0]@x
    q+= u.T@theta[1].T@theta[1]@u
    q+= 2*x.T@theta[2]@u + theta[3]
    return q[0,0]



## Initialize environemnt

Below, we initialize the environment that you need to interact with in this excercise.

In [ ]:
n = 2
m = 2
env = LQEnv(n,m)

### (a) Assuming knowledge of the system matrices $A, B, R,$ and $Q$, compute $P$, the solution to the appropriate form of the Riccati equation, and the matrix $K$ that characterizes the optimal policy. Evaluate the policy performance over 100 trajectories each with length 1000.


In [ ]:
### Your code goes here


### (b) Given an arbitrary $\Theta_0$, write the greedy policy with respect to $Q(x; u; \Theta_0)$ in terms of the parameters $\Theta_0$. What is the form of this greedy policy?

*type your answer here*

### (c) Implement Q-learning for this problem, where actions are chosen using the greedy policy (wrt to the current Q-function). Then train using samples chosen from a single arbitrarily long trajectory. Use the given function to initialize theta.

Hint: If your states/parameters grow very large during training, consider making the learning rate very small

In [ ]:
def init_theta(n,m, fun = np.random.randn, seed = 1):
    np.random.seed(seed)
    A = fun(n,n)
    B = fun(m,m)
    C = fun(n,m)
    const = 0
    return [A,B,C, const]



In [ ]:
# Your Code goes HERE

### Do the parameters converge? Is the resulting policy the same as the policy obtained in part (a)? If not, why?


*Type your answer here*

### (d) Train using  samples chosen from multiple trajectories. Each with length 25.  Also choose actions using an $\epsilon$-greedy policy for $\epsilon = 0.3$. Do the parameters converge? Is the resulting policy the same as the policy obtained in part (a)? Evaluate the policy performance over 100 trajectories each with length 1000 and compare its average cost with that of the optimal policy.

Hint: to sample actions randomly, use the method: env.sample_random_action()

In [ ]:
# Your code goes here

### Do the parameters converge? Is the resulting policy the same as the policy obtained in part (a)?


*type your answer here*

### Evaluate the policy performance over 100 trajectories each with length 1000 and compare its average cost with that of the optimal policy.


In [ ]:
# Your code goes here

### (e)  Try different values of epsilons $\in \{0.2,0.4,0.6, 0.8\}$ (using the same initialization of $\Theta$). With which value of epsilon does the algorithm converges faster?



In [ ]:
# Your code goes here

###  (f) Is it fair to generalize the conclusions we have found about the optimal way to select epsilon and the trajectory length to other problems/environment? For example, if you have found that a single infinite trajectory does not work well, and larger values of epsilon works better, can you generalize this to other environments/problems like, for example, chess? why/why not?

*type your answer here*